In [1]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [2]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

    
val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header", false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

raw_sentiment.groupBy($"label").count.show

+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+



dataSchema = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment = [label: int, tweet: string]


[label: int, tweet: string]

In [3]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")

val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

val rf = new RandomForestClassifier()
    .setNumTrees(10)
    .setMaxDepth(5)
    .setSeed(42)

val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, rf))

tokenizer = tok_990673c8c7a2
hashingTF = hashingTF_7d5836592905
rf = rfc_842ac936a364
pipeline = pipeline_9dba2f215e93


pipeline_9dba2f215e93

In [4]:
val model = pipeline.fit(raw_sentiment)

model = pipeline_9dba2f215e93


pipeline_9dba2f215e93

In [5]:
model.write.overwrite().save("/home/jovyan/models/spark-ml-model")

In [6]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

sameModel = pipeline_9dba2f215e93


pipeline_9dba2f215e93

In [7]:
val predictionsDF = sameModel.transform(raw_sentiment)

predictionsDF.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|@switchfoot http:...|[@switchfoot, htt...|(1000,[7,14,21,54...|[4.65386113846161...|[0.46538611384616...|       1.0|
|    0|is upset that he ...|[is, upset, that,...|(1000,[170,193,22...|[4.94029737402610...|[0.49402973740261...|       1.0|
|    0|@Kenichan I dived...|[@kenichan, i, di...|(1000,[10,36,77,1...|[5.16229218307118...|[0.51622921830711...|       0.0|
|    0|my whole body fee...|[my, whole, body,...|(1000,[82,191,296...|[4.78237740305735...|[0.47823774030573...|       1.0|
|    0|@nationwideclass ...|[@nationwideclass...|(1000,[18,96,130,...|[5.17020120329489...|[0.51702012032948...|       0.0|
|    0|@

predictionsDF = [label: int, tweet: string ... 5 more fields]


[label: int, tweet: string ... 5 more fields]

In [8]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [9]:
predictionsDF.select(getProbability($"probability").alias("clean_probability")).show

+-------------------+
|  clean_probability|
+-------------------+
| 0.5346138861538383|
| 0.5059702625973896|
| 0.4837707816928815|
| 0.5217622596942644|
| 0.4829798796705109|
| 0.5004873694026386|
| 0.5259401762003582|
| 0.4938168520086216|
| 0.5157563715281381|
| 0.5217622596942644|
| 0.5217622596942644|
| 0.4920798462889506|
| 0.4837707816928815|
|0.47735458966428973|
| 0.4497997521221035|
|0.46402182643447487|
| 0.5134531950981953|
| 0.5134531950981953|
|  0.534547762351379|
| 0.5387450342438135|
+-------------------+
only showing top 20 rows

